In [65]:
import pandas as pd
import glob, os

L'obiettivo di questo script è creare un dataset con le differenze delle statistiche tra squadra di casa e squadra di trasferta, in base alle statistiche medie

In [66]:
merged_matches_by_home = pd.read_csv("SerieA/Season21_22/matches.csv", index_col=0)

In [67]:
columns = ['matchday', 'date', 'h_team', 'a_team', 'result']
matches = merged_matches_by_home[columns]

In [68]:
diff_stats = ['goals', 'total_shots', 'shots_on_target', 'goals_on_penalty', 'total_penalties', 'completed_passings', 'total_passings', 'corners', 'percentage_possession', 'fouls', 'yellow_cards', 'red_cards']    

In [69]:
stats_of_all_teams = {}
for file in sorted(os.listdir("SerieA/Season21_22/AvgStats2/")):
    if file.endswith(".csv"):
        team_name = file.replace('.csv', '')
        team_stats = pd.read_csv("SerieA/Season21_22/AvgStats2/"+file, index_col=0) 
        stats_of_all_teams[team_name] = team_stats

In [70]:
matches = matches.reset_index(drop=True)
matches[:2]

,matchday,date,h_team,a_team,result
0,Giornata n. 1,2021-08-21,Hellas Verona,Sassuolo,P
1,Giornata n. 1,2021-08-21,Inter,Genoa,V


Effettua il match in base alla squadra e alla data della partita.
Essendo tutte le partite salvate in dataframe e ciascun dataframe è il valore di un dizionario, con chiave il nome della squadra, utilizziamo il nome della squadra per cercare il valore del dizionario

In [75]:
def find_row(team, date):
    matches_played_by_team = stats_of_all_teams[team]
    match_found = matches_played_by_team[matches_played_by_team.date==date]
    match_found = match_found.drop(columns=['team', 'date', 'matchday', 'Risultato'])
    return match_found

In [124]:
matches_with_diffs = pd.DataFrame()

for id, match in matches.iterrows():
    home, away, date = match.h_team, match.a_team, match.date

    #ottengo i dataset con le statistiche delle 2 squadre (home ed away)
    home_stats = find_row(home, date) 
    away_stats = find_row(away, date)
    
    tmp = match
    
    #per ciascuna colonna nei due dataframe (le colonne sono le stesse), calcolo la differenza tra statistica_home e statistica_away
    for col in home_stats.columns: 
        valueH = home_stats[col].values[0]
        valueA = away_stats[col].values[0]
        dif = valueH - valueA
        tmp[col] = dif
    tmp = tmp.to_frame().T
    
    if matches_with_diffs.empty:
        matches_with_diffs = tmp
    else: 
        matches_with_diffs = matches_with_diffs.append(tmp, ignore_index=True)

In [ ]:
matches_with_diffs.to_csv('SerieA/Season21_22/diffs_dataset.csv')